# **Experiment Hub:**


        



In [3]:
import sys
# install dependencies
%pip install -r ../req.txt;

Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2 -> 24.0
[notice] To update, run: C:\Users\robby\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ERROR: Could not open requirements file: [Errno 2] No such file or directory: '../req.txt;'


In [4]:
# experiment overview:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox, Layout, HBox, Output
from IPython.display import Markdown, display, clear_output, HTML
import sys
from pathlib import Path

# Adjust the system path to include the pythonScripts directory using a relative path
python_scripts_path = Path('..') / 'pythonScripts'
sys.path.append(str(python_scripts_path.resolve()))

from new_experiment import create_experiment

# Define project options and styles
project_options = ['', 'TPH_NO', 'TPH_Fe(IV)O','BesC']
experiment_options = ['', 'AbsCD', 'MCD']
flexoki_button_style = {'button_color': '#6BA9F0', 'text_color': '#0F0E0E', 'font_weight': 'bold'}
flexoki_text_style = {'description_width': '100px', 'background':'#0F0E0E', 'font-family': 'arial', 'text_color': '#ffffff'}
# Apply dark theme styling with specified colors
dark_theme_css = """
<style>
.jupyter-widgets .widget-label { color: #EDEEE3; }  /* Make the description text white */
.widget-dropdown > select { background-color: #0F0E0E !important; color: #EDEEE3; }
textarea, input { font-family: Arial; }
.cell-output-ipywidget-background { background-color: transparent !important; }
.jp-OutputArea-output { background-color: transparent; }

/* Styles for Markdown tables */
table { 
    color: white; /* Text color */
    border-collapse: collapse;
}
td, th { 
    /* Table border */
    padding: 12px; 
    text-align: left;
}
tr:nth-child(even){background-color: #0F0E0E;}
tr:hover {background-color: #252423;}
th {
    padding-top: 12px;
    padding-bottom: 12px;
    background-color: #0F0E0E; /* Header background color */
    color: #6BA9F0; /* Header text color */
    border-bottom: 1px solid white;
}
</style>
"""
display(HTML(dark_theme_css))  # Apply the CSS

# Output widgets for different UI components
filter_output = Output()
df_output = Output()
add_experiment_output = Output()

# Initialize the filter_project_widget globally
filter_project_widget_unformatted = widgets.Dropdown(
    description="Filter Project:",
    options=project_options,
    style={'description_width': '100px'}
)

# Create a container with layout settings to align its content to the right
filter_project_widget = HBox([filter_project_widget_unformatted], layout=Layout(justify_content='flex-end'))

# Display output widgets
display(filter_output)
display(df_output)
display(add_experiment_output)

# Function to display the DataFrame as a markdown table
def csv_to_markdown_table(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # # Parse the 'Date' column to datetime format assuming the format is 'dd-mm-yy'
    # df['Date'] = pd.to_datetime(df['Date'], format='%y-%m-%d')
    
    # Sort the DataFrame by the 'Date' column in descending order
    df = df.sort_values(by='Date', ascending=False)
    
    # Convert the DataFrame to a markdown table string
    markdown_str = df.to_markdown(index=False)
    
    # Display the markdown table
    with df_output:
        clear_output(wait=True)
        display(Markdown(markdown_str))

# Function to filter the DataFrame based on project selection
def filter_df():
    df = pd.read_csv(csv_file_path)  # Re-read the CSV file every time the function is called
    with filter_output:
        clear_output(wait=True)
        display(filter_project_widget)  # Use the global widget

    def on_filter_change(change):
        filtered_df = df[df['Project'] == change['new']] if change['new'] else df
        filtered_df = filtered_df.sort_values(by='Date', ascending=False)
        markdown_str = filtered_df.to_markdown(index=False)
        with df_output:
            clear_output(wait=True)
            display(Markdown(markdown_str))

    filter_project_widget_unformatted.observe(on_filter_change, names='value')

# Interactive UI for adding a new experiment
def interactive_add_experiment():
    with add_experiment_output:
        clear_output(wait=True)  # Clear previous UI components if any

        # Widgets for input fields
        experiment_id_widget = widgets.Text(description="Experiment ID:", style=flexoki_text_style)
        project_widget = widgets.Dropdown(description="Project:", options=project_options)
        type_widget = widgets.Dropdown(description="Type:", options=experiment_options)
        description_widget = widgets.Textarea(description="Description:", style=flexoki_text_style)
        add_button = widgets.Button(description="Add Experiment", style=flexoki_button_style)

        # Organize widgets into layout
        form = VBox([HBox([experiment_id_widget, project_widget],layout=Layout(display='flex', justify_content='flex-end', padding='0px')),
                     HBox([description_widget, type_widget],layout=Layout(display='flex', justify_content='flex-end', padding='0px')),
                     HBox([add_button], layout=Layout(display='flex', justify_content='flex-end', padding='0px'))],
                    layout=Layout(padding='20px'))

        def on_add_button_clicked(b):
            # Check if there is a space in the exp_id string:
            experiment_id = experiment_id_widget.value.replace(" ", "")
            # Create a new experiment entry
            create_experiment(experiment_id, type_widget.value, project_widget.value, description_widget.value)
            # Reset the filter dropdown to its default value
            filter_project_widget_unformatted.value = ''
            experiment_id_widget.value = ''
            type_widget.value = ''
            project_widget.value = ''
            description_widget.value = ''
            # Refresh the displayed CSV content and filter
            csv_to_markdown_table(csv_file_path)
            filter_df()  # Refresh the filter

        add_button.on_click(on_add_button_clicked)
        display(form)

# Adjust the path to the CSV file using a relative path
csv_file_path = Path('experiments.csv').resolve()

# Initial setup: display CSV content, setup filter UI, and add experiment UI
csv_to_markdown_table(csv_file_path)
filter_df()
interactive_add_experiment()



Output()

Output()

Output()